In [1]:
from src.dataloader import create_dataloaders_ER
import pandas as pd

In [2]:
model_name_or_path="../model/deberta-v3-base/" ## modify it with your own path
task = "ER"
noise_ratio=0
batch_size=32
rank = 16
target_modules=["query_proj", "key_proj", "value_proj"] ## you can select ["value_proj", "query_proj", "key_proj"] for the target_modules for deberta fine-tuning
device="cuda:0" ## Only support single-GPU
num_epochs=20
lr= 3e-5

dataloader_outputs = create_dataloaders_ER(model_name_or_path=model_name_or_path,
                                           task=task,
                                           noise_ratio=noise_ratio,
                                           batch_size=batch_size,
                                           train_file = '/data/home/wangys/geil/PyG_Dataset/Hospital/detector/train.csv',
                                           valid_file = '/data/home/wangys/geil/PyG_Dataset/Hospital/detector/valid.csv',
                                           test_file = '/data/home/wangys/geil/PyG_Dataset/Hospital/detector/test.csv',
                                           max_length = 256)
train_dataloader, eval_dataloader,test_dataloader, noise_index, tokenized_datasets, collate_fn = dataloader_outputs


Index(['0', '1', '2'], dtype='object') Index(['0', '1', '2'], dtype='object') Index(['0', '1', '2'], dtype='object')


Map:   0%|          | 0/2028 [00:00<?, ? examples/s]

Map:   0%|          | 0/2028 [00:00<?, ? examples/s]

Map:   0%|          | 0/20000 [00:00<?, ? examples/s]

In [4]:
import pandas as pd
result = pd.read_csv('/data/home/wangys/transfer-er/Pipeline/LLM_output/--Amazon-Google-Test-Select-P1-Eval.csv',index_col=0)

In [5]:
result

,instruction,input,output,reject_list,ltable_id,rtable_id,predict
0,You are an expert in detecting if two text des...,NaN,{'index': 1},"[2756, 2109, 2452, 2321, 2603, 2593]",1162,2109,{'index': 1}
1,You are an expert in detecting if two text des...,NaN,{'index': 0},"[358, 2860, 465, 176, 1575, 3188]",741,358,{'index': 3}
2,You are an expert in detecting if two text des...,NaN,{'index': 0},"[2816, 3091, 2497, 1473, 3027, 915]",587,2816,{'index': 0}
3,You are an expert in detecting if two text des...,NaN,{'index': 0},"[2585, 1593, 2560, 2540, 2567, 671]",809,2585,{'index': 0}
4,You are an expert in detecting if two text des...,NaN,{'index': 0},"[813, 532, 159, 2636, 1501, 819]",1012,813,{'index': 0}
...,...,...,...,...,...,...,...
1162,You are an expert in detecting if two text des...,NaN,{'index': 0},"[256, 2876, 542, 871, 549, 835]",674,256,{'index': 3}
1163,You are an expert in detecting if two text des...,NaN,{'index': 1},"[2121, 1445, 1282, 2099, 1407, 2988]",25,1445,{'index': 1}
1164,You are an expert in detecting if two text des...,NaN,{'index': 0},"[2309, 2123, 2872, 317, 563, 435]",886,2309,{'index': 0}
1165,You are an expert in detecting if two text des...,NaN,{'index': 0},"[920, 912, 985, 1396, 325, 2003]",1149,920,{'index': 2}


In [1]:
# Please install OpenAI SDK first: `pip3 install openai`

from openai import OpenAI

client = OpenAI(api_key="sk-f21d19cc479b4b18a966f6d4f1f497b1", base_url="https://api.deepseek.com")

response = client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": "You are a helpful assistant"},
        {"role": "user", "content": "Hello"},
    ],
    stream=False
)

print(response.choices[0].message.content)

APITimeoutError: Request timed out.

In [3]:
from src.lora_model import LORAEngineDeberta,LORAEngineDebertaMultiClass
lora_engine = LORAEngineDeberta(model_name_or_path=model_name_or_path,
                            target_modules=target_modules,
                            train_dataloader=train_dataloader,
                            eval_dataloader=eval_dataloader, ## 这行代码可以直接查看模型的prediction性能
                            test_dataloader=test_dataloader,
                            device=device,
                            num_epochs=num_epochs,
                            lr=lr,
                            lora=False,
                            low_rank=rank,
                            task=task,
                            num_labels=2,
                            valid_each_epoch=False,
                            use_rslora=False
                            ) ## num_labels may vary from specific tasks

[2025-06-16 23:10:39,346] [INFO] [real_accelerator.py:239:get_accelerator] Setting ds_accelerator to cuda (auto detect)


/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: cannot find -laio: No such file or directory
collect2: error: ld returned 1 exit status
/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: warning: librt.so.1, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: warning: libpthread.so.0, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: warning: libstdc++.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: warning: libm.so.6, needed by /usr/local/cuda/lib64/libcufile.so, not found (try using -rpath or -rpath-link)
/data/home/wangys/anaconda3/envs/verl/compiler_compat/ld: /usr/local/cuda/lib64/libcufile.so: undefined reference to `std::runtime_error::~runtime_error()@GLIBCXX_3.4'
/d

In [4]:
lora_engine.build_LORA_model()
lora_engine.train_LORA_model()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at ../model/deberta-v3-base/ and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/64 [00:00<?, ?it/s]

You're using a DebertaV2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch 1: Training Loss = 0.6804339340887964


  0%|          | 0/64 [00:00<?, ?it/s]

...

Epoch 19: Training Loss = 0.05496740283297186


  0%|          | 0/64 [00:00<?, ?it/s]

Epoch 20: Training Loss = 0.0546070744167082


  0%|          | 0/625 [00:00<?, ?it/s]

Prediction Result on Test Data: {'f1': 0.764638346727899}
